In [1]:
import pandas as pd
import numpy as np

In [2]:
zhi = pd.read_csv('data/zhi_zip.csv')
files = ['CRMLSSold202406_filled.csv', 'CRMLSSold202407_filled.csv', 'CRMLSSold202408.csv', 
         'CRMLSSold202409.csv', 'CRMLSSold202410.csv', 'CRMLSSold202411.csv', 'CRMLSSold202412.csv', 
         'CRMLSSold202501_filled.csv', 'CRMLSSold202502.csv', 'CRMLSSold202503.csv']

FileNotFoundError: [Errno 2] No such file or directory: 'data/zhi_zip.csv'

In [5]:
combined = pd.read_csv(f'data/{files[0]}')

for i in range(1, len(files)):
    next_file = pd.read_csv(f'data/{files[i]}')
    if 'latfilled' in next_file:
        next_file = next_file.drop(columns=['latfilled', 'lonfilled'])
    combined = pd.concat([combined, next_file], axis=0)

combined

,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,MainLevelBedrooms,NewConstructionYN,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict,latfilled,lonfilled
0,RanchoSoutheast,RanchoSoutheast,NaN,True,NaN,NaN,True,1250.0,543227709,pat@patowen.com,...,0.0,False,0.0,Menifee Union,92587,0.0,12197.0,NaN,True,True
1,BeverlyHillsGreaterLA,BeverlyHillsGreaterLA,NaN,True,NaN,NaN,NaN,2100000.0,531525691,andrew@epicpropertygroup.com,...,NaN,False,NaN,NaN,91206,NaN,437284.0,NaN,False,False
2,JoshuaTreeGateway,JoshuaTreeGateway,NaN,True,NaN,NaN,NaN,179000.0,531017430,bryan@jtmodern.com,...,NaN,False,NaN,NaN,92252,0.0,875556.0,NaN,True,True
3,HighDesert,HighDesert,NaN,True,NaN,NaN,NaN,14500.0,530411639,trustdee@aol.com,...,NaN,False,NaN,NaN,92345,NaN,871200.0,NaN,True,True
4,Southland,Southland,NaN,True,NaN,NaN,NaN,720000.0,526412917,karen@goldenoakrealty.com,...,NaN,False,NaN,NaN,93510,0.0,2576054.0,NaN,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21440,SanDiego,SanDiego,Wood,True,NaN,NaN,False,2495.0,1013509844,mannybehar@yahoo.com,...,NaN,NaN,0.0,NaN,92117,0.0,NaN,NaN,NaN,NaN
21441,SanLuisObispoCoastal,SanLuisObispoCoastal,NaN,True,NaN,NaN,NaN,15000.0,1013508981,jack@jackposemsky.com,...,NaN,False,NaN,NaN,93428,0.0,4233.0,NaN,NaN,NaN
21442,CitrusValley,CitrusValley,NaN,NaN,NaN,NaN,NaN,1600000.0,1013260692,bdkallen@gmail.com,...,NaN,NaN,NaN,NaN,92345,NaN,69107.0,NaN,NaN,NaN
21443,Laguna,Laguna,"Carpet,Tile,Wood",True,NaN,NaN,False,40000.0,1002662608,andrea@ballesterosgroup.com,...,1.0,False,2.0,Laguna Beach Unified,92651,0.0,6560.0,NaN,NaN,NaN


In [10]:
zhi['RegionName'].iloc[0]

77494

In [11]:
print(len(zhi['RegionName'].unique()))
combined.shape[0]

def fill_zero(zip):
    zip = str(zip)
    if len(zip) == 4:
        return '0' + zip
    return zip

zhi['RegionName'] = zhi['RegionName'].transform(fill_zero)

26212


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2024-05-31,2024-06-30,2024-07-31,2024-08-31,2024-09-30,2024-10-31,2024-11-30,2024-12-31,2025-01-31,2025-02-28
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,209828.020780,...,496253.244801,496553.107777,496174.295064,496582.857048,497274.251498,498400.299687,499123.504118,499546.968696,499973.511292,499654.680546
1,61148,2,08701,zip,NJ,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,158061.973712,...,722916.533955,729583.546922,735041.118394,740709.964353,747957.057909,755078.269921,760258.639100,763119.734611,764217.885821,767356.052713
2,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,103597.115830,...,281941.076475,281904.311804,281672.609759,281493.561519,281307.557564,280893.863104,280110.305022,279385.886181,278779.853771,278158.652309
3,62080,4,11368,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,298395.178403,...,888586.448255,889148.984784,887807.323801,887096.643675,889634.415009,891761.307355,892945.746907,893022.204406,892420.023545,892924.437131
4,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,102565.496503,...,275833.372840,275663.404320,275369.718067,275102.956977,274858.795713,274360.845549,273769.040379,273436.101153,273200.206676,272804.441919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26207,71617,39992,31648,zip,GA,GA,NaN,"Valdosta, GA",Echols County,NaN,...,76159.527821,76026.315825,74704.704477,72956.542980,71193.758352,69924.013695,69290.730486,68986.995867,68614.248456,67338.899230
26208,93941,39992,82515,zip,WY,WY,Hudson,"Riverton, WY",Fremont County,NaN,...,175493.088412,175849.775658,175297.304327,175131.139673,175022.733487,175349.799018,174997.106431,174756.223194,174505.424601,174557.400720
26209,63527,39992,14441,zip,NY,NY,Dresden,"Rochester, NY",Yates County,94063.345314,...,211331.699473,216131.403312,219152.168443,221454.754360,223600.881405,225778.351391,227009.737070,226940.446234,225864.812444,223847.004492
26210,99927,39992,98628,zip,WA,WA,Klickitat,NaN,Klickitat County,NaN,...,186753.806129,187443.160462,185270.404482,182773.946190,180509.543090,179368.461683,178261.692258,177419.507118,176017.319209,174681.345856


In [20]:
averaged = zhi.groupby('RegionName')['2025-02-28'].mean().to_frame()
averaged = averaged.rename(columns = {'2025-02-28': 'mean_zhi'})
averaged.to_csv('data/mean_zhi.csv')

In [22]:
averaged = averaged.reset_index()

In [26]:
combined['PostalCode'] = combined['PostalCode'].astype(str)
data_w_zhi = pd.merge(combined, averaged, how='left', left_on='PostalCode', right_on='RegionName')
data_w_zhi

,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict,latfilled,lonfilled,RegionName,mean_zhi
0,RanchoSoutheast,RanchoSoutheast,NaN,True,NaN,NaN,True,1250.0,543227709,pat@patowen.com,...,0.0,Menifee Union,92587,0.0,12197.0,NaN,True,True,92587,6.438523e+05
1,BeverlyHillsGreaterLA,BeverlyHillsGreaterLA,NaN,True,NaN,NaN,NaN,2100000.0,531525691,andrew@epicpropertygroup.com,...,NaN,NaN,91206,NaN,437284.0,NaN,False,False,91206,1.502375e+06
2,JoshuaTreeGateway,JoshuaTreeGateway,NaN,True,NaN,NaN,NaN,179000.0,531017430,bryan@jtmodern.com,...,NaN,NaN,92252,0.0,875556.0,NaN,True,True,92252,3.639486e+05
3,HighDesert,HighDesert,NaN,True,NaN,NaN,NaN,14500.0,530411639,trustdee@aol.com,...,NaN,NaN,92345,NaN,871200.0,NaN,True,True,92345,4.444423e+05
4,Southland,Southland,NaN,True,NaN,NaN,NaN,720000.0,526412917,karen@goldenoakrealty.com,...,NaN,NaN,93510,0.0,2576054.0,NaN,True,True,93510,8.633323e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219067,SanDiego,SanDiego,Wood,True,NaN,NaN,False,2495.0,1013509844,mannybehar@yahoo.com,...,0.0,NaN,92117,0.0,NaN,NaN,NaN,NaN,92117,1.153098e+06
219068,SanLuisObispoCoastal,SanLuisObispoCoastal,NaN,True,NaN,NaN,NaN,15000.0,1013508981,jack@jackposemsky.com,...,NaN,NaN,93428,0.0,4233.0,NaN,NaN,NaN,93428,9.958060e+05
219069,CitrusValley,CitrusValley,NaN,NaN,NaN,NaN,NaN,1600000.0,1013260692,bdkallen@gmail.com,...,NaN,NaN,92345,NaN,69107.0,NaN,NaN,NaN,92345,4.444423e+05
219070,Laguna,Laguna,"Carpet,Tile,Wood",True,NaN,NaN,False,40000.0,1002662608,andrea@ballesterosgroup.com,...,2.0,Laguna Beach Unified,92651,0.0,6560.0,NaN,NaN,NaN,92651,3.136573e+06


In [35]:
redfin = pd.read_csv('data/median_redfin.csv')
redfin = redfin.iloc[:, 1:]
redfin['region'] = redfin['region'].astype(str)
complete_data = pd.merge(data_w_zhi, redfin, how='left', left_on='PostalCode', right_on='region')
complete_data = complete_data.drop(columns=['region', 'RegionName'])
complete_data = complete_data[(complete_data['PropertyType'] == 'Residential') & 
                              (complete_data['PropertySubType'] == 'SingleFamilyResidence')]

,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict,latfilled,lonfilled,mean_zhi,median_sale_price
0,RanchoSoutheast,RanchoSoutheast,NaN,True,NaN,NaN,True,1250.0,543227709,pat@patowen.com,...,0.0,Menifee Union,92587,0.0,12197.0,NaN,True,True,6.438523e+05,375000.0
1,BeverlyHillsGreaterLA,BeverlyHillsGreaterLA,NaN,True,NaN,NaN,NaN,2100000.0,531525691,andrew@epicpropertygroup.com,...,NaN,NaN,91206,NaN,437284.0,NaN,False,False,1.502375e+06,700000.0
2,JoshuaTreeGateway,JoshuaTreeGateway,NaN,True,NaN,NaN,NaN,179000.0,531017430,bryan@jtmodern.com,...,NaN,NaN,92252,0.0,875556.0,NaN,True,True,3.639486e+05,188750.0
3,HighDesert,HighDesert,NaN,True,NaN,NaN,NaN,14500.0,530411639,trustdee@aol.com,...,NaN,NaN,92345,NaN,871200.0,NaN,True,True,4.444423e+05,259750.0
4,Southland,Southland,NaN,True,NaN,NaN,NaN,720000.0,526412917,karen@goldenoakrealty.com,...,NaN,NaN,93510,0.0,2576054.0,NaN,True,True,8.633323e+05,589250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219067,SanDiego,SanDiego,Wood,True,NaN,NaN,False,2495.0,1013509844,mannybehar@yahoo.com,...,0.0,NaN,92117,0.0,NaN,NaN,NaN,NaN,1.153098e+06,562500.0
219068,SanLuisObispoCoastal,SanLuisObispoCoastal,NaN,True,NaN,NaN,NaN,15000.0,1013508981,jack@jackposemsky.com,...,NaN,NaN,93428,0.0,4233.0,NaN,NaN,NaN,9.958060e+05,650000.0
219069,CitrusValley,CitrusValley,NaN,NaN,NaN,NaN,NaN,1600000.0,1013260692,bdkallen@gmail.com,...,NaN,NaN,92345,NaN,69107.0,NaN,NaN,NaN,4.444423e+05,259750.0
219070,Laguna,Laguna,"Carpet,Tile,Wood",True,NaN,NaN,False,40000.0,1002662608,andrea@ballesterosgroup.com,...,2.0,Laguna Beach Unified,92651,0.0,6560.0,NaN,NaN,NaN,3.136573e+06,1769000.0


In [3]:
import zipfile
zip_path = 'data/updated_data.csv.zip'

with zipfile.ZipFile(zip_path, 'r') as z:
    with z.open("updated_data.csv") as f:  # Change "yourfile.csv" to the actual filename
        df = pd.read_csv(f)

df = df.iloc[:, 1:]

/var/folders/r6/j2xslqfj0lg0bxy1nj8b6fzm0000gn/T/ipykernel_14493/3165546398.py:6: DtypeWarning: Columns (5,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [4]:
df = df[(df['PropertyType'] == 'Residential') & (df['PropertySubType'] == 'SingleFamilyResidence')]
df

,BuyerAgentAOR,ListAgentAOR,Flooring,ViewYN,WaterfrontYN,BasementYN,PoolPrivateYN,OriginalListPrice,ListingKey,ListAgentEmail,...,GarageSpaces,HighSchoolDistrict,PostalCode,AssociationFee,LotSizeSquareFeet,MiddleOrJuniorSchoolDistrict,latfilled,lonfilled,mean_zhi,median_sale_price
23,MLSListings,MLSListings,"Laminate,Tile",False,NaN,NaN,NaN,NaN,1077427852,bholmes@compass.com,...,1.0,Other,94025,NaN,7500.0,NaN,True,True,2.939172e+06,1893750.0
24,SouthBay,SouthBay,NaN,True,NaN,NaN,True,7750000.0,1077370123,TeamLyon@LyonStahl.com,...,2.0,Hermosa,90254,0.0,7051.0,NaN,True,True,2.299881e+06,1665000.0
25,MLSListings,MLSListings,NaN,False,NaN,NaN,NaN,NaN,1077294226,mbarrera0615@gmail.com,...,1.0,Other,93906,NaN,7600.0,NaN,True,True,7.068663e+05,385000.0
27,OutofAreaBoard,OutofAreaBoard,Wood,False,NaN,NaN,False,1810000.0,1077271803,EdanAmar@yahoo.com,...,NaN,NaN,90046,NaN,6205.0,NaN,False,False,1.949203e+06,1268500.0
29,SanDiego,SanDiego,NaN,False,NaN,NaN,False,1135000.0,1077148727,doug@dougwestgroup.com,...,3.0,NaN,92057,150.0,6378.0,NaN,True,True,8.509078e+05,409250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219019,Fresno,Fresno,"Laminate,Tile",True,NaN,True,False,899000.0,1038736627,KirstenEnglund@gmail.com,...,2.0,Yosemite Unified,93669,350.0,13068.0,NaN,NaN,NaN,3.731014e+05,253250.0
219024,WestSanGabrielValley,WestSanGabrielValley,NaN,True,NaN,NaN,False,339000.0,1038169240,realestate@marcos-nunez.com,...,0.0,Rim of the World,92325,0.0,3236.0,NaN,NaN,NaN,3.852381e+05,225000.0
219035,Southland,Southland,"Carpet,Laminate",False,NaN,NaN,False,2600000.0,1035525173,jamal@jfkrealty.com,...,2.0,Palos Verdes Peninsula Unified,90066,0.0,5700.0,NaN,NaN,NaN,1.755851e+06,992125.0
219036,Arcadia,Arcadia,NaN,True,NaN,NaN,True,5880000.0,1035522392,frank.ferguson@kw.com,...,5.0,Santa Paula,93015,0.0,1947132.0,NaN,NaN,NaN,7.018057e+05,441000.0


In [5]:
df['ClosePrice']

23        3000000.0
24        7750000.0
25         575000.0
27        1810000.0
29        1135000.0
            ...    
219019     650000.0
219024     275000.0
219035    2225000.0
219036    3737500.0
219037     375000.0
Name: ClosePrice, Length: 110533, dtype: float64

In [6]:
df.to_csv('data/updated_data.csv')